### lending club data preprocessing 
#### Purpose
Preparing lending club data for binary class classification
#### overview
    1. only keep rows with target classes ('Charged Off' and 'Fully Paid')
    2. remove 
        2.1 columns with unique value
        2.2 columns contain more than 40% missing values
        2.3 columns which may cause informatoin leakage
        2.4 columns provide redundant information
        2.5 rows with too many missing values
    3. Categorical variable
        3.1 remove variables with too many levels
        3.2 dummy coding
    4. fill missing values using Multivariate imputer
    5. data normalization
    6. train test spliting 

In [1]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('max_columns', 100)
pd.set_option('max_colwidth', 5000) #show more information of columns

import numpy as np
from sklearn import preprocessing #data normalization 
from sklearn.experimental import enable_iterative_imputer #missing value imputation 
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split

### data info
Since loans in Lending Club are either 36 or 60 months, and in order to have a good number of finished loans, loan data issued from 2011 to 2014 were downloaded. And in the datasets after 2015, many of the loan status are 'current' and not finished.

In [2]:
df = pd.read_csv("LendingClub_LoanStats_2014.csv")
df.sample(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,...,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
73368,27552291,NaN,14500.0,14500.0,14500.0,36 months,12.99%,488.50,C,C1,Research Coordinator,7 years,RENT,45000.0,Not Verified,Sep-14,Charged Off,n,https://lendingclub.com/browse/loanDetail.action?loan_id=27552291,NaN,debt_consolidation,Debt consolidation,850xx,AZ,33.52,2.0,Jun-98,690.0,694.0,1.0,14.0,NaN,9.0,0.0,13295.0,85.80%,36.0,f,0.0,0.0,6848.47000,6848.47,3481.63,1403.37,0.0,1963.47,353.4246,Aug-15,488.50,NaN,...,9.0,6.0,9.0,0.0,0.0,0.0,1.0,96.8,100.0,0.0,0.0,166309.0,143617.0,12400.0,150909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
139787,18506237,NaN,6850.0,6850.0,6850.0,36 months,10.15%,221.52,B,B2,Controller,3 years,MORTGAGE,110000.0,Source Verified,Jun-14,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=18506237,NaN,debt_consolidation,Debt consolidation,352xx,AL,26.34,0.0,Sep-00,685.0,689.0,0.0,NaN,NaN,20.0,0.0,61559.0,59.20%,41.0,w,0.0,0.0,6907.94000,6907.94,6850.00,57.94,0.0,0.00,0.0000,Jul-14,6907.94,NaN,...,17.0,11.0,20.0,0.0,0.0,0.0,4.0,100.0,40.0,0.0,0.0,460486.0,143591.0,92500.0,136180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
97242,23914923,NaN,4900.0,4900.0,4900.0,36 months,14.99%,169.84,C,C5,Administration,10+ years,RENT,70000.0,Source Verified,Aug-14,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=23914923,NaN,debt_consolidation,Debt consolidation,100xx,NY,16.56,0.0,Mar-82,695.0,699.0,0.0,NaN,NaN,12.0,0.0,32845.0,88.10%,14.0,f,0.0,0.0,6114.06476,6114.06,4900.00,1214.06,0.0,0.00,0.0000,Aug-17,169.66,NaN,...,11.0,9.0,12.0,0.0,0.0,0.0,1.0,100.0,83.3,0.0,0.0,95606.0,85340.0,31300.0,58306.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
234423,10069958,NaN,13000.0,13000.0,13000.0,60 months,16.99%,323.02,D,D1,E-5,5 years,RENT,50000.0,Verified,Jan-14,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=10069958,NaN,credit_card,Credit Refinance,921xx,CA,6.79,0.0,Jun-05,680.0,684.0,2.0,56.0,NaN,5.0,0.0,15049.0,43.40%,14.0,f,0.0,0.0,18117.53379,18117.53,13000.00,5117.53,0.0,0.00,0.0000,Dec-16,7134.85,NaN,...,11.0,2.0,5.0,0.0,0.0,0.0,1.0,78.6,NaN,0.0,0.0,34700.0,15049.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN

In [3]:
df.shape

(235631, 150)

There are 235631 observations and 150 features in this dataset

### round1: target feature
There are 193878 Fully Paid (good) loans and 41748 Charged Off (bad) loans, since the purpose of this project is to classify good and bad loans, other loan status types will be removed at this step.

In [4]:
df['loan_status'].value_counts()

Fully Paid            193878
Charged Off            41748
Current                    1
Late (31-120 days)         1
Default                    1
Name: loan_status, dtype: int64

In [5]:
df1 = df[(df['loan_status'] == "Fully Paid") | (df['loan_status'] == "Charged Off")]
df1.shape

(235626, 150)

### round2: remove columns with unique value
Columns with only one value provide no help on classification.

In [6]:
df1Columns = df1.columns
dropUniqueColumns = []
for col in df1Columns:
    if len(df1[col].unique()) == 1:
        dropUniqueColumns.append(col)
        
df2 = df1.drop(dropUniqueColumns, axis=1)

print("number of columns with unique values: " + str(len(dropUniqueColumns)))
dropUniqueColumns

number of columns with unique values: 37


['member_id',
 'pymnt_plan',
 'out_prncp',
 'out_prncp_inv',
 'next_pymnt_d',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'revol_bal_joint',
 'sec_app_fico_range_low',
 'sec_app_fico_range_high',
 'sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
 'sec_app_mths_since_last_major_derog']

### round3: remove features with more than 40% missing values

In [7]:
df2Columns = df2.columns
dropNullColumns = []
for col in df2Columns:
    colNullPercent = (df2[col].isna().sum())/len(df2[col])
    if colNullPercent > 0.40:
        dropNullColumns.append(col)

df3 = df2.drop(dropNullColumns, axis=1)

print("number of columns with missing value more than 40%: " + str(len(dropNullColumns)))
dropNullColumns

number of columns with missing value more than 40%: 26


['desc',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'mths_since_last_major_derog',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_revol_delinq',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'deferral_term',
 'hardship_amount',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_length',
 'hardship_dpd',
 'hardship_loan_status',
 'orig_projected_additional_accrued_interest',
 'hardship_payoff_balance_amount',
 'hardship_last_payment_amount',
 'debt_settlement_flag_date',
 'settlement_status',
 'settlement_date',
 'settlement_amount',
 'settlement_percentage',
 'settlement_term']

#### take a look at the remaining data
after first 3 rounds, number of features are reduced from 150 to 87. And some of them still have missing value, will deal with them later.

In [8]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235626 entries, 0 to 235628
Data columns (total 87 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          235626 non-null  object 
 1   loan_amnt                   235626 non-null  float64
 2   funded_amnt                 235626 non-null  float64
 3   funded_amnt_inv             235626 non-null  float64
 4   term                        235626 non-null  object 
 5   int_rate                    235626 non-null  object 
 6   installment                 235626 non-null  float64
 7   grade                       235626 non-null  object 
 8   sub_grade                   235626 non-null  object 
 9   emp_title                   222390 non-null  object 
 10  emp_length                  223607 non-null  object 
 11  home_ownership              235626 non-null  object 
 12  annual_inc                  235626 non-null  float64
 13  verification_s

### round 4: mannually remove features based on Dictionary
It is needed to remove features which may cause information leakage of the final result. Since this project's purpose is to predict whether a loan will be charged off **before** issueing a loan, such features may include information that happended **after** a loan was issued. There are also many features provide similar information which are redundant and also need to be removed.

* Prepare a dataset which contains feature name, feature type, a value of the feature and the feature's description.

In [9]:
dictionary = pd.read_csv("LCDataDictionary.csv",skipfooter=1) #skip last row
dictionary.sample(3)

,LoanStatNew,Description,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
20,emp_title,The job title supplied by the Borrower when applying for the loan.*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,sec_app_revol_util,Ratio of total current balance to high credit/credit limit for all revolving accounts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,total_acc,The total number of credit lines currently in the borrower's credit file,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
dictionary = dictionary.rename(columns={'LoanStatNew': 'name'})
dictionary = dictionary[['name','Description']]

df3Types = pd.DataFrame(df3.dtypes,columns=['dtypes']).reset_index()
df3Types['name'] = df3Types['index']
df3Types = df3Types[['name','dtypes']]
df3Types['first value'] = df3.loc[0].values

preview = df3Types.merge(dictionary, on='name',how='left')
preview.tail()

,name,dtypes,first value,Description
82,total_bal_ex_mort,float64,15030,Total credit balance excluding mortgage
83,total_bc_limit,float64,13000,Total bankcard high credit/credit limit
84,total_il_high_credit_limit,float64,11325,Total installment high credit/credit limit
85,hardship_flag,object,N,Flags whether or not the borrower is on a hardship plan
86,debt_settlement_flag,object,N,"Flags whether or not the borrower, who has charged-off, is working with a debt-settlement company."


* Since there are too much features and it is hard to figure out features at once, they are divided into four groups.

##### round4.1 group1 

In [11]:
preview[:22]

,name,dtypes,first value,Description
0,id,object,36805548,A unique LC assigned ID for the loan listing.
1,loan_amnt,float64,10400,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
2,funded_amnt,float64,10400,The total amount committed to that loan at that point in time.
3,funded_amnt_inv,float64,10400,The total amount committed by investors for that loan at that point in time.
4,term,object,36 months,The number of payments on the loan. Values are in months and can be either 36 or 60.
5,int_rate,object,6.99%,Interest Rate on the loan
6,installment,float64,321.08,The monthly payment owed by the borrower if the loan originates.
7,grade,object,A,LC assigned loan grade
8,sub_grade,object,A3,LC assigned loan subgrade
9,emp_title,object,Truck Driver Delivery Personel,The job title supplied by the Borrower when applying for the loan.*


* **id, url**: useless for modeling
* **funded_amnt, funded_amnt_inv, issue_d**: happend after loan issuing 
* **grade, sub_grade** redundant information as they are based on interest rate
* **emp_title, zip_code, addr_state**: too much levels based on experience

In [12]:
drop_list = ['id','funded_amnt','funded_amnt_inv','grade','sub_grade',
             'emp_title','issue_d','url','zip_code','addr_state']
df4_1 = df3.drop(drop_list,axis=1)
df4_1.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10400.0,36 months,6.99%,321.08,8 years,MORTGAGE,58000.0,Not Verified,Charged Off,credit_card,Credit card refinancing,14.92,0.0,Sep-89,710.0,714.0,2.0,17.0,0.0,6133.0,31.60%,36.0,w,6611.69000,6611.69,5217.75,872.67,0.0,521.27,93.8286,Aug-16,321.08,Feb-17,564.0,560.0,0.0,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,1.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,N,N
1,15000.0,60 months,12.39%,336.64,10+ years,RENT,78000.0,Source Verified,Fully Paid,debt_consolidation,Debt consolidation,12.03,0.0,Aug-94,750.0,754.0,0.0,6.0,0.0,138008.0,29%,17.0,w,17392.37000,17392.37,15000.00,2392.37,0.0,0.00,0.0000,Jun-16,12017.81,Jul-20,714.0,710.0,0.0,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,N,N
2,9600.0,36 months,13.66%,326.53,10+ years,RENT,69000.0,Source Verified,Fully Paid,debt_consolidation,Debt consolidation,25.81,0.0,Nov-92,680.0,684.0,0.0,12.0,0.0,16388.0,59.40%,44.0,f,9973.43000,9973.43,9600.00,373.43,0.0,0.00,0.0000,Apr-15,9338.58,Jul-20,714.0,710.0,0.0,0.0,0.0,38566.0,27600.0,8.0,3214.0,6494.0,69.2,0.0,0.0,183.0,265.0,23.0,3.0,0.0,24.0,17.0,0.0,4.0,7.0,5.0,16.0,17.0,8.0,26.0,7.0,12.0,0.0,0.0,0.0,3.0,100.0,60.0,0.0,0.0,52490.0,38566.0,21100.0,24890.0,N,N
3,12800.0,60 months,17.14%,319.08,10+ years,MORTGAGE,125000.0,Verified,Fully Paid,car,Car financing,8.31,1.0,Oct-00,665.0,669.0,0.0,8.0,0.0,5753.0,100.90%,13.0,w,19165.35192,19165.35,12800.00,6365.35,0.0,0.00,0.0000,Sep-19,1576.08,Apr-20,704.0,700.0,0.0,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,368700.0,18007.0,4400.0,18000.0,N,N
4,21425.0,60 months,15.59%,516.36,6 years,RENT,63800.0,Source Verified,Fully Paid,credit_card,Credit card refinancing,18.49,0.0,Aug-03,685.0,689.0,0.0,10.0,0.0,16374.0,76.20%,35.0,w,25512.20000,25512.20,21425.00,4087.20,0.0,0.00,0.0000,May-16,17813.19,Apr-18,529.0,525.0,0.0,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0,N,N


##### round4.2 group2

In [13]:
preview[22:44]

,name,dtypes,first value,Description
22,delinq_2yrs,float64,0,The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years
23,earliest_cr_line,object,Sep-89,The month the borrower's earliest reported credit line was opened
24,fico_range_low,float64,710,The lower boundary range the borrower�s FICO at loan origination belongs to.
25,fico_range_high,float64,714,The upper boundary range the borrower�s FICO at loan origination belongs to.
26,inq_last_6mths,float64,2,The number of inquiries in past 6 months (excluding auto and mortgage inquiries)
27,open_acc,float64,17,The number of open credit lines in the borrower's credit file.
28,pub_rec,float64,0,Number of derogatory public records
29,revol_bal,float64,6133,Total credit revolving balance
30,revol_util,object,31.60%,"Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit."
31,total_acc,float64,36,The total number of credit lines currently in the borrower's credit file


* **total_pymnt, total_pymnt_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt**: happend after loan issuing 

In [14]:
drop_list = ['total_pymnt','total_pymnt_inv',
             'total_rec_prncp', 
             'total_rec_int','total_rec_late_fee',
             'recoveries','collection_recovery_fee',
             'last_pymnt_d','last_pymnt_amnt']
df4_2 = df4_1.drop(drop_list,axis=1)
df4_2.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10400.0,36 months,6.99%,321.08,8 years,MORTGAGE,58000.0,Not Verified,Charged Off,credit_card,Credit card refinancing,14.92,0.0,Sep-89,710.0,714.0,2.0,17.0,0.0,6133.0,31.60%,36.0,w,Feb-17,564.0,560.0,0.0,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,1.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,N,N
1,15000.0,60 months,12.39%,336.64,10+ years,RENT,78000.0,Source Verified,Fully Paid,debt_consolidation,Debt consolidation,12.03,0.0,Aug-94,750.0,754.0,0.0,6.0,0.0,138008.0,29%,17.0,w,Jul-20,714.0,710.0,0.0,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,N,N
2,9600.0,36 months,13.66%,326.53,10+ years,RENT,69000.0,Source Verified,Fully Paid,debt_consolidation,Debt consolidation,25.81,0.0,Nov-92,680.0,684.0,0.0,12.0,0.0,16388.0,59.40%,44.0,f,Jul-20,714.0,710.0,0.0,0.0,0.0,38566.0,27600.0,8.0,3214.0,6494.0,69.2,0.0,0.0,183.0,265.0,23.0,3.0,0.0,24.0,17.0,0.0,4.0,7.0,5.0,16.0,17.0,8.0,26.0,7.0,12.0,0.0,0.0,0.0,3.0,100.0,60.0,0.0,0.0,52490.0,38566.0,21100.0,24890.0,N,N
3,12800.0,60 months,17.14%,319.08,10+ years,MORTGAGE,125000.0,Verified,Fully Paid,car,Car financing,8.31,1.0,Oct-00,665.0,669.0,0.0,8.0,0.0,5753.0,100.90%,13.0,w,Apr-20,704.0,700.0,0.0,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,368700.0,18007.0,4400.0,18000.0,N,N
4,21425.0,60 months,15.59%,516.36,6 years,RENT,63800.0,Source Verified,Fully Paid,credit_card,Credit card refinancing,18.49,0.0,Aug-03,685.0,689.0,0.0,10.0,0.0,16374.0,76.20%,35.0,w,Apr-18,529.0,525.0,0.0,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0,N,N


##### round4.3 group3

In [15]:
preview[44:66]

,name,dtypes,first value,Description
44,last_fico_range_low,float64,560,The lower boundary range the borrower�s last FICO pulled belongs to.
45,collections_12_mths_ex_med,float64,0,Number of collections in 12 months excluding medical collections
46,acc_now_delinq,float64,0,The number of accounts on which the borrower is now delinquent.
47,tot_coll_amt,float64,0,Total collection amounts ever owed
48,tot_cur_bal,float64,162110,Total current balance of all accounts
49,total_rev_hi_lim,float64,19400,NaN
50,acc_open_past_24mths,float64,7,Number of trades opened in past 24 months.
51,avg_cur_bal,float64,9536,Average current balance of all accounts
52,bc_open_to_buy,float64,7599,Total open to buy on revolving bankcards.
53,bc_util,float64,41.5,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


##### round4.4 group4

In [16]:
preview[66:]

,name,dtypes,first value,Description
66,num_bc_sats,float64,7,Number of satisfactory bankcard accounts
67,num_bc_tl,float64,18,Number of bankcard accounts
68,num_il_tl,float64,2,Number of installment accounts
69,num_op_rev_tl,float64,14,Number of open revolving accounts
70,num_rev_accts,float64,32,Number of revolving accounts
71,num_rev_tl_bal_gt_0,float64,9,Number of revolving trades with balance >0
72,num_sats,float64,17,Number of satisfactory accounts
73,num_tl_120dpd_2m,float64,0,Number of accounts currently 120 days past due (updated in past 2 months)
74,num_tl_30dpd,float64,0,Number of accounts currently 30 days past due (updated in past 2 months)
75,num_tl_90g_dpd_24m,float64,0,Number of accounts 90 or more days past due in last 24 months


* no features need to remove after round 4.3 and 4.4

### round5 missing value
* There are 68 features remaining

In [17]:
df4_2.shape

(235626, 68)

* Keep only the rows with at least 66 non-NA values (remove rows with 2 or more missing values)

In [18]:
df5 = df4_2.dropna(thresh=66) 
df5.shape

(232885, 68)

* check remaining missing value

In [19]:
null_counts = df5.isnull().sum()
print(null_counts[null_counts>0])

emp_length               11571
revol_util                  23
last_credit_pull_d          24
bc_util                    159
mo_sin_old_il_acct        6887
mths_since_recent_inq    21177
num_tl_120dpd_2m          7738
percent_bc_gt_75            83
hardship_flag                1
dtype: int64


* revol_util, last_credit_pull_d, etc only have few missing values, remove rows have missing values in them

In [20]:
df5 = df5.dropna(subset=['revol_util','last_credit_pull_d','bc_util','percent_bc_gt_75','hardship_flag'])
null_counts = df5.isnull().sum()
print(null_counts[null_counts>0])

emp_length               11561
mo_sin_old_il_acct        6884
mths_since_recent_inq    21167
num_tl_120dpd_2m          7647
dtype: int64


### round 6.1 Categorical variables

'object' type may contain categorical variables.

In [21]:
object_columns_df = df5.select_dtypes(include=['object'])
print(object_columns_df.iloc[0])

term                                  36 months
int_rate                                  6.99%
emp_length                              8 years
home_ownership                         MORTGAGE
verification_status                Not Verified
loan_status                         Charged Off
purpose                             credit_card
title                   Credit card refinancing
earliest_cr_line                         Sep-89
revol_util                               31.60%
initial_list_status                           w
last_credit_pull_d                       Feb-17
hardship_flag                                 N
debt_settlement_flag                          N
Name: 0, dtype: object


* remove **earliest_cr_line** and **last_credit_pull_d** because they are date info and are useless for modeling
* correct **int_rate** and **revol_util** to float type

In [22]:
df6_1 = df5.drop(['earliest_cr_line','last_credit_pull_d'], axis=1)
for col in ['int_rate','revol_util']:
    df6_1[col] = df6_1[col].str.rstrip('%').astype('float')

* check distribution of categorical variables 

In [23]:
cols = ['loan_status','term','emp_length','home_ownership','verification_status',
        'purpose','title','initial_list_status','hardship_flag','debt_settlement_flag']
for col in cols:
    print(df6_1[col].value_counts(dropna=False))
    print("-------")

Fully Paid     191439
Charged Off     41179
Name: loan_status, dtype: int64
-------
 36 months    160328
 60 months     72290
Name: term, dtype: int64
-------
10+ years    78446
2 years      20282
3 years      18087
< 1 year     17804
1 year       14449
4 years      13384
7 years      12951
5 years      12920
8 years      11726
6 years      11675
NaN          11561
9 years       9333
Name: emp_length, dtype: int64
-------
MORTGAGE    118392
RENT         91573
OWN          22652
ANY              1
Name: home_ownership, dtype: int64
-------
Source Verified    98736
Not Verified       67622
Verified           66260
Name: verification_status, dtype: int64
-------
debt_consolidation    141268
credit_card            55111
home_improvement       12738
other                  10106
major_purchase          3781
medical                 2265
small_business          2236
car                     1799
moving                  1294
vacation                1153
house                    738
renewable_ene

##### remove
*  **title** has too much levels 
*  **title** only have one level
*  **home_ownership** with level 'ANY'

In [24]:
df6_1 = df6_1.drop(['title','hardship_flag'], axis=1)
df6_1 = df6_1.drop(df6_1[df6_1['home_ownership'] == 'ANY'].index) 

##### map
*  **emp_length** to ordinal numbers
*  **loan_status** to 0, 1

In [25]:
mapping_dict = {
    'emp_length':{
        '10+ years': 10,
        '9 years': 9,
        '8 years': 8,
        '7 years': 7,
        '6 years': 6,
        '5 years': 5,
        '4 years': 4,
        '3 years': 3,
        '2 years': 2,
        '1 year': 1,
        '< 1 year': 0
    },
    'loan_status':{
        'Fully Paid':0,
        'Charged Off':1
    }
}

df6_1 = df6_1.replace(mapping_dict)
df6_1.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,debt_settlement_flag
0,10400.0,36 months,6.99,321.08,8.0,MORTGAGE,58000.0,Not Verified,1,credit_card,14.92,0.0,710.0,714.0,2.0,17.0,0.0,6133.0,31.6,36.0,w,564.0,560.0,0.0,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,1.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,N
1,15000.0,60 months,12.39,336.64,10.0,RENT,78000.0,Source Verified,0,debt_consolidation,12.03,0.0,750.0,754.0,0.0,6.0,0.0,138008.0,29.0,17.0,w,714.0,710.0,0.0,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,N
2,9600.0,36 months,13.66,326.53,10.0,RENT,69000.0,Source Verified,0,debt_consolidation,25.81,0.0,680.0,684.0,0.0,12.0,0.0,16388.0,59.4,44.0,f,714.0,710.0,0.0,0.0,0.0,38566.0,27600.0,8.0,3214.0,6494.0,69.2,0.0,0.0,183.0,265.0,23.0,3.0,0.0,24.0,17.0,0.0,4.0,7.0,5.0,16.0,17.0,8.0,26.0,7.0,12.0,0.0,0.0,0.0,3.0,100.0,60.0,0.0,0.0,52490.0,38566.0,21100.0,24890.0,N
3,12800.0,60 months,17.14,319.08,10.0,MORTGAGE,125000.0,Verified,0,car,8.31,1.0,665.0,669.0,0.0,8.0,0.0,5753.0,100.9,13.0,w,704.0,700.0,0.0,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,368700.0,18007.0,4400.0,18000.0,N
4,21425.0,60 months,15.59,516.36,6.0,RENT,63800.0,Source Verified,0,credit_card,18.49,0.0,685.0,689.0,0.0,10.0,0.0,16374.0,76.2,35.0,w,529.0,525.0,0.0,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0,N


###### dummy coding

In [26]:
cat_columns = ['term','home_ownership','verification_status',
               'purpose','initial_list_status','debt_settlement_flag']
dummy_df = pd.get_dummies(df6_1[cat_columns])
df6_1 = pd.concat([df6_1, dummy_df], axis=1)
df6_1 = df6_1.drop(cat_columns, axis=1)
df6_1.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w,debt_settlement_flag_N,debt_settlement_flag_Y
0,10400.0,6.99,321.08,8.0,58000.0,1,14.92,0.0,710.0,714.0,2.0,17.0,0.0,6133.0,31.6,36.0,564.0,560.0,0.0,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,1.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,15000.0,12.39,336.64,10.0,78000.0,0,12.03,0.0,750.0,754.0,0.0,6.0,0.0,138008.0,29.0,17.0,714.0,710.0,0.0,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,9600.0,13.66,326.53,10.0,69000.0,0,25.81,0.0,680.0,684.0,0.0,12.0,0.0,16388.0,59.4,44.0,714.0,710.0,0.0,0.0,0.0,38566.0,27600.0,8.0,3214.0,6494.0,69.2,0.0,0.0,183.0,265.0,23.0,3.0,0.0,24.0,17.0,0.0,4.0,7.0,5.0,16.0,17.0,8.0,26.0,7.0,12.0,0.0,0.0,0.0,3.0,100.0,60.0,0.0,0.0,52490.0,38566.0,21100.0,24890.0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,12800.0,17.14,319.08,10.0,125000.0,0,8.31,1.0,665.0,669.0,0.0,8.0,0.0,5753.0,100.9,13.0,704.0,700.0,0.0,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,368700.0,18007.0,4400.0,18000.0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,21425.0,15.59,516.36,6.0,63800.0,0,18.49,0.0,685.0,689.0,0.0,10.0,0.0,16374.0,76.2,35.0,529.0,525.0,0.0,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


### round6.2 continuous variables

In [27]:
float_columns_df = df6_1.select_dtypes(include=['float64'])
print(float_columns_df.iloc[0])

loan_amnt                      10400.00
int_rate                           6.99
installment                      321.08
emp_length                         8.00
annual_inc                     58000.00
dti                               14.92
delinq_2yrs                        0.00
fico_range_low                   710.00
fico_range_high                  714.00
inq_last_6mths                     2.00
open_acc                          17.00
pub_rec                            0.00
revol_bal                       6133.00
revol_util                        31.60
total_acc                         36.00
last_fico_range_high             564.00
last_fico_range_low              560.00
collections_12_mths_ex_med         0.00
acc_now_delinq                     0.00
tot_coll_amt                       0.00
tot_cur_bal                   162110.00
total_rev_hi_lim               19400.00
acc_open_past_24mths               7.00
avg_cur_bal                     9536.00
bc_open_to_buy                  7599.00


* continues variables contain different range of data, outliers may be a potential problem, will deal with it later

* all the remaining missing value are numerical variables more specifically, integers. Multivarite imputer are used to impute them.

In [28]:
null_counts = df6_1.isnull().sum()
print(null_counts[null_counts>0])

emp_length               11561
mo_sin_old_il_acct        6884
mths_since_recent_inq    21167
num_tl_120dpd_2m          7647
dtype: int64


In [29]:
cols = null_counts[null_counts>0].index
df6_1[cols].sample(10)

,emp_length,mo_sin_old_il_acct,mths_since_recent_inq,num_tl_120dpd_2m
63380,8.0,94.0,4.0,0.0
216546,10.0,161.0,1.0,0.0
199689,10.0,116.0,7.0,0.0
92398,10.0,231.0,NaN,0.0
1719,10.0,146.0,23.0,0.0
78897,8.0,97.0,12.0,0.0
172228,10.0,113.0,6.0,0.0
87129,7.0,NaN,2.0,0.0
138427,2.0,22.0,8.0,0.0
62134,10.0,219.0,6.0,0.0


In [30]:
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(df6_1)
IterativeImputer(random_state=0)
df_filled = imp.transform(df6_1)

In [31]:
df_filled = pd.DataFrame(df_filled, columns = df6_1.columns)
df_filled[cols] = np.round(df_filled[cols])
df_filled.sample(10)

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w,debt_settlement_flag_N,debt_settlement_flag_Y
53256,27000.0,10.15,873.12,10.0,70000.0,0.0,28.89,0.0,695.0,699.0,1.0,30.0,0.0,41781.0,51.0,45.0,644.0,640.0,0.0,0.0,0.0,71948.0,81145.0,10.0,2767.0,5338.0,71.0,0.0,0.0,199.0,272.0,6.0,6.0,2.0,6.0,5.0,0.0,9.0,24.0,15.0,15.0,5.0,24.0,39.0,16.0,26.0,-0.0,0.0,0.0,5.0,100.0,44.4,0.0,0.0,152316.0,48482.0,48040.0,13971.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
174325,35000.0,19.99,1300.55,5.0,127000.0,0.0,15.30,0.0,680.0,684.0,0.0,12.0,0.0,32031.0,86.8,23.0,534.0,530.0,0.0,0.0,0.0,315103.0,36900.0,4.0,28646.0,3556.0,89.7,0.0,0.0,82.0,138.0,21.0,4.0,3.0,21.0,8.0,0.0,7.0,8.0,8.0,14.0,5.0,9.0,15.0,8.0,12.0,0.0,0.0,0.0,1.0,100.0,85.7,0.0,0.0,329252.0,65860.0,34400.0,42070.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
51766,24000.0,10.99,521.70,9.0,550006.0,0.0,2.47,0.0,705.0,709.0,0.0,8.0,0.0,30058.0,92.2,27.0,704.0,700.0,0.0,0.0,0.0,47584.0,32600.0,3.0,5948.0,2542.0,92.2,0.0,0.0,290.0,248.0,16.0,2.0,3.0,16.0,10.0,0.0,5.0,5.0,5.0,5.0,14.0,5.0,10.0,5.0,8.0,0.0,0.0,0.0,1.0,100.0,80.0,0.0,0.0,56059.0,47584.0,32600.0,23459.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
154037,13000.0,10.99,425.55,5.0,42000.0,0.0,25.91,0.0,695.0,699.0,1.0,8.0,0.0,13724.0,42.8,11.0,584.0,580.0,0.0,0.0,0.0,24755.0,32100.0,3.0,3094.0,18376.0,42.8,0.0,0.0,85.0,105.0,3.0,3.0,0.0,3.0,4.0,0.0,4.0,4.0,4.0,6.0,5.0,4.0,6.0,4.0,8.0,0.0,0.0,0.0,1.0,100.0,75.0,0.0,0.0,55536.0,24755.0,32100.0,23436.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
177534,24000.0,12.49,539.83,10.0,69000.0,1.0,25.95,0.0,700.0,704.0,0.0,14.0,0.0,47952.0,63.7,25.0,559.0,555.0,0.0,0.0,0.0,208562.0,75300.0,4.0,16043.0,22648.0,67.9,0.0,0.0,46.0,179.0,17.0,6.0,4.0,17.0,14.0,0.0,8.0,8.0,8.0,12.0,4.0,10.0,16.0,8.0,14.0,0.0,0.0,0.0,1.0,100.0,25.0,0.0,0.0,255097.0,58636.0,70600.0,16914.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
180948,7000.0,9.67,224.79,8.0,47000.0,0.0,7.05,0.0,710.0,714.0,0.0,9.0,1.0,8584.0,37.5,17.0,754.0,750.0,0.0,0.0,0.0,111495.0,22900.0,2.0,12388.0,276.0,86.2,0.0,0.0,167.0,146.0,14.0,14.0,2.0,15.0,11.0,0.0,2.0,6.0,2.0,2.0,2.0,8.0,13.0,6.0,9.0,0.0,0.0,0.0,0.0,100.0,100.0,1.0,0.0,132080.0,8584.0,2000.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
164750,8000.0,9.67,256.90,10.0,35000.0,0.0,7.24,0.0,670.0,674.0,0.0,7.0,0.0,4745.0,59.3,20.0,769.0,765.0,1.0,0.0,689.0,54145.0,

* no missing value remaining 

In [32]:
null_counts = df_filled.isnull().sum()
print(null_counts[null_counts>0])
print(df_filled.shape)

Series([], dtype: int64)
(232617, 83)


### save data

In [33]:
df_filled.to_csv("LendingClub_2011_2014_cleanedData.csv")

###### identify target attr and perform class mapping

In [39]:
Class_mapping={label:idx for idx,label in enumerate(np.unique(df_filled['loan_status']))}
print(Class_mapping)
df_filled['loan_status']=df_filled['loan_status'].map(Class_mapping)
df_filled.head()

{0: 0, 1: 1}


,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w,debt_settlement_flag_N,debt_settlement_flag_Y
0,10400.0,6.99,321.08,8.0,58000.0,1,14.92,0.0,710.0,714.0,2.0,17.0,0.0,6133.0,31.6,36.0,564.0,560.0,0.0,0.0,0.0,162110.0,19400.0,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,1.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,15000.0,12.39,336.64,10.0,78000.0,0,12.03,0.0,750.0,754.0,0.0,6.0,0.0,138008.0,29.0,17.0,714.0,710.0,0.0,0.0,0.0,149140.0,184500.0,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,9.0,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,9600.0,13.66,326.53,10.0,69000.0,0,25.81,0.0,680.0,684.0,0.0,12.0,0.0,16388.0,59.4,44.0,714.0,710.0,0.0,0.0,0.0,38566.0,27600.0,8.0,3214.0,6494.0,69.2,0.0,0.0,183.0,265.0,23.0,3.0,0.0,24.0,17.0,0.0,4.0,7.0,5.0,16.0,17.0,8.0,26.0,7.0,12.0,0.0,0.0,0.0,3.0,100.0,60.0,0.0,0.0,52490.0,38566.0,21100.0,24890.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,12800.0,17.14,319.08,10.0,125000.0,0,8.31,1.0,665.0,669.0,0.0,8.0,0.0,5753.0,100.9,13.0,704.0,700.0,0.0,0.0,0.0,261815.0,5700.0,2.0,32727.0,0.0,103.2,0.0,0.0,16.0,170.0,21.0,16.0,5.0,21.0,1.0,1.0,3.0,5.0,3.0,5.0,1.0,5.0,7.0,5.0,8.0,0.0,0.0,0.0,0.0,76.9,100.0,0.0,0.0,368700.0,18007.0,4400.0,18000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,21425.0,15.59,516.36,6.0,63800.0,0,18.49,0.0,685.0,689.0,0.0,10.0,0.0,16374.0,76.2,35.0,529.0,525.0,0.0,0.0,0.0,42315.0,21500.0,4.0,4232.0,324.0,97.8,0.0,0.0,135.0,136.0,7.0,7.0,0.0,7.0,7.0,1.0,3.0,4.0,3.0,12.0,16.0,5.0,18.0,4.0,10.0,0.0,0.0,0.0,2.0,91.4,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


### normalization
###### normalize all remaining attributes to (0,1) range

In [40]:
num=df_filled.drop('loan_status', axis=1)
x=num.values
min_max_scaler=preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)
num_scaled=min_max_scaler.fit_transform(x)
num_norm=pd.DataFrame(num_scaled,columns=num.columns)
num_norm.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w,debt_settlement_flag_N,debt_settlement_flag_Y
0,0.276471,0.049352,0.214708,0.714286,0.007336,0.373093,0.000000,0.270270,0.268817,0.333333,0.192771,0.0,0.002395,0.086197,0.220779,0.663529,0.662722,0.0,0.0,0.0,0.042207,0.00192,0.132075,0.019168,0.029199,0.162618,0.0,0.0,0.135472,0.341289,0.002688,0.004425,0.027027,0.008117,0.04,0.121212,0.230769,0.236842,0.200000,0.283333,0.013333,0.213115,0.291262,0.236842,0.192771,0.0,0.0,0.0,0.153846,0.799520,0.143,0.0,0.0,0.017911,0.005590,0.011828,0.009120,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.411765,0.318544,0.225929,0.857143,0.010004,0.300825,0.000000,0.486486,0.483871,0.000000,0.060241,0.0,0.053895,0.079105,0.097403,0.840000,0.840237,0.0,0.0,0.0,0.038831,0.01843,0.094340,0.059958,0.036599,0.018417,0.0,0.0,0.183601,0.286396,0.002688,0.004425,0.000000,0.076299,0.36,0.000000,0.038462,0.105263,0.028571,0.016667,0.053333,0.065574,0.067961,0.105263,0.060241,0.0,0.0,0.0,0.153846,1.000000,0.000,0.0,0.0,0.019621,0.055465,0.009078,0.009664,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.252941,0.381854,0.218638,0.857143,0.008804,0.645411,0.000000,0.108108,0.107527,0.000000,0.132530,0.0,0.006400,0.162029,0.272727,0.840000,0.840237,0.0,0.0,0.0,0.010041,0.00274,0.150943,0.006461,0.024953,0.271160,0.0,0.0,0.326203,0.311456,0.061828,0.013274,0.000000,0.038961,0.68,0.000000,0.153846,0.184211,0.142857,0.250000,0.113333,0.114754,0.233010,0.184211,0.132530,0.0,0.0,0.0,0.115385,1.000000,0.600,0.0,0.0,0.005219,0.014343,0.019255,0.020044,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.347059,0.555334,0.213265,0.857143,0.016273,0.207802,0.045455,0.027027,0.026882,0.000000,0.084337,0.0,0.002247,0.275232,0.071429,0.828235,0.828402,0.0,0.0,0.0,0.068167,0.00055,0.037736,0.065785,0.000000,0.404389,0.0,0.0,0.028520,0.198091,0.056452,0.070796,0.135135,0.034091,0.04,0.030303,0.115385,0.131579,0.085714,0.066667,0.006667,0.065574,0.048544,0.131579,0.084337,0.0,0.0,0.0,0.000000,0.722689,1.000,0.0,0.0,0.036841,0.006697,0.003943,0.014495,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.600735,0.478066,0.355538,0.571429,0.008110,0.462366,0.000000,0.135135,0.134409,0.000000,0.108434,0.0,0.006394,0.207856,0.214286,0.622353,0.621302,0.0,0.0,0.0,0.011017,0.00213,0.075472,0.008507,0.001245,0.383229,0.0,0.0,0.240642,0.157518,0.018817,0.030973,0.000000,0.011364,0.28,0.030303,0.115385,0.105263,0.085714,0.183333,0.106667,0.065574,0.155340,0.105263,0.108434,0.0,0.0,0.0,0.076923,0.896759,

In [36]:
dfNormalized=pd.merge(df_filled['loan_status'],num_norm,left_index=True, right_index=True)
dfNormalized.to_csv("loan_normalized",index=False)
dfNormalized.head()

,loan_status,loan_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w,debt_settlement_flag_N,debt_settlement_flag_Y
0,1,0.276471,0.049352,0.214708,0.714286,0.007336,0.373093,0.000000,0.270270,0.268817,0.333333,0.192771,0.0,0.002395,0.086197,0.220779,0.663529,0.662722,0.0,0.0,0.0,0.042207,0.00192,0.132075,0.019168,0.029199,0.162618,0.0,0.0,0.135472,0.341289,0.002688,0.004425,0.027027,0.008117,0.04,0.121212,0.230769,0.236842,0.200000,0.283333,0.013333,0.213115,0.291262,0.236842,0.192771,0.0,0.0,0.0,0.153846,0.799520,0.143,0.0,0.0,0.017911,0.005590,0.011828,0.009120,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0,0.411765,0.318544,0.225929,0.857143,0.010004,0.300825,0.000000,0.486486,0.483871,0.000000,0.060241,0.0,0.053895,0.079105,0.097403,0.840000,0.840237,0.0,0.0,0.0,0.038831,0.01843,0.094340,0.059958,0.036599,0.018417,0.0,0.0,0.183601,0.286396,0.002688,0.004425,0.000000,0.076299,0.36,0.000000,0.038462,0.105263,0.028571,0.016667,0.053333,0.065574,0.067961,0.105263,0.060241,0.0,0.0,0.0,0.153846,1.000000,0.000,0.0,0.0,0.019621,0.055465,0.009078,0.009664,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0,0.252941,0.381854,0.218638,0.857143,0.008804,0.645411,0.000000,0.108108,0.107527,0.000000,0.132530,0.0,0.006400,0.162029,0.272727,0.840000,0.840237,0.0,0.0,0.0,0.010041,0.00274,0.150943,0.006461,0.024953,0.271160,0.0,0.0,0.326203,0.311456,0.061828,0.013274,0.000000,0.038961,0.68,0.000000,0.153846,0.184211,0.142857,0.250000,0.113333,0.114754,0.233010,0.184211,0.132530,0.0,0.0,0.0,0.115385,1.000000,0.600,0.0,0.0,0.005219,0.014343,0.019255,0.020044,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0,0.347059,0.555334,0.213265,0.857143,0.016273,0.207802,0.045455,0.027027,0.026882,0.000000,0.084337,0.0,0.002247,0.275232,0.071429,0.828235,0.828402,0.0,0.0,0.0,0.068167,0.00055,0.037736,0.065785,0.000000,0.404389,0.0,0.0,0.028520,0.198091,0.056452,0.070796,0.135135,0.034091,0.04,0.030303,0.115385,0.131579,0.085714,0.066667,0.006667,0.065574,0.048544,0.131579,0.084337,0.0,0.0,0.0,0.000000,0.722689,1.000,0.0,0.0,0.036841,0.006697,0.003943,0.014495,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0,0.600735,0.478066,0.355538,0.571429,0.008110,0.462366,0.000000,0.135135,0.134409,0.000000,0.108434,0.0,0.006394,0.207856,0.214286,0.622353,0.621302,0.0,0.0,0.0,0.011017,0.00213,0.075472,0.008507,0.001245,0.383229,0.0,0.0,0.240642,0.157518,0.018817,0.030973,0.000000,0.011364,0.28,0.030303,0.115385,0.105263,0.085714,0.183333,0.106667,0.065574,0.155340,0.105263,0.108434,0.0,0.0,

### train test split
###### split data into 2/3 training and 1/3 testing dataset

In [37]:
x = dfNormalized.drop('loan_status', axis=1)
y = dfNormalized['loan_status']
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=1/3, random_state=0, stratify=y)
print(x_train.shape[0], x_test.shape[0], y_train.shape[0], y_test.shape[0])

155078 77539 155078 77539


###### save training and test data

In [38]:
x_train.to_csv("x_train.csv",index=False)
x_test.to_csv("x_test.csv",index=False)
y_train.to_csv("y_train.csv",index=False)
y_test.to_csv("y_test.csv",index=False)